In [19]:
import pdfplumber
import re
import json
import os
import PyPDF2

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict

In [31]:
file_path = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
load_dotenv()

True

In [13]:
# una semplice estrazione dei capitoli dal pdf
"""def extract_chapters_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"

    # Suddividiamo il testo nei capitoli basandoci su "CAPITOLO X"
    chapters = re.split(r'(Capitolo \d+)', text)
    
    structured_data = {}
    for i in range(1, len(chapters), 2):
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        structured_data[chapter_title] = chapter_content
    
    return structured_data

# Leggiamo il PDF e estraiamo i capitoli
pdf_chapters = extract_chapters_from_pdf(file_path)

# Stampiamo i titoli dei capitoli
print("📖 Capitoli trovati:")
for chapter in pdf_chapters.keys():
    print(f"- {chapter}")
"""

📖 Capitoli trovati:
- Capitolo 1
- Capitolo 2
- Capitolo 3
- Capitolo 4
- Capitolo 5


In [ ]:
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

In [46]:
class TechniqueExtractionAgent:
    def __init__(self, api_key: str, api_base: str):
        """
        Inizializza l'agente di estrazione con Azure OpenAI.
        """
        self.llm = AzureChatOpenAI(
            openai_api_version="2024-08-01-preview",
            azure_deployment="o1-mini",
            azure_endpoint=api_base,
            api_key=api_key,
            temperature=1  # Permettiamo più creatività per ottenere risultati più flessibili
        )
        
        # Prompt ancora più rilassato
        self.extraction_template = """
            Sei un assistente esperto nell'analisi di testi culinari.
            Analizza il seguente testo e cerca di identificare qualsiasi possibile tecnica culinaria.
            
            Restituisci un elenco delle tecniche trovate, anche se non strutturate perfettamente.
            
            Esempio di risposta:
            ```json
            {{
                "tecniche": [
                    "nome tecnica 1",
                    "nome tecnica 2",
                    "nome tecnica 3"
                ]
            }}
            ```
            
            Se non trovi informazioni precise, prova comunque a restituire concetti utili.
            
            Testo da analizzare:
            {text}
            
            Rispondi SOLO con il JSON, senza altro testo.
        """
        
        self.prompt = ChatPromptTemplate.from_template(self.extraction_template)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=4000,  # Aumentiamo ulteriormente il contesto
            chunk_overlap=500
        )

    def extract_text_from_pdf(self, pdf_path: str, chapters: List[int]) -> Dict[int, str]:
        """
        Estrae il testo dai capitoli specificati del PDF.
        """
        chapter_texts = {}
        
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            current_chapter = None
            current_text = []
            
            for page in pdf_reader.pages:
                text = page.extract_text()
                
                for chapter_num in chapters:
                    if f"Capitolo {chapter_num}" in text:
                        if current_chapter is not None:
                            chapter_texts[current_chapter] = " ".join(current_text)
                        current_chapter = chapter_num
                        current_text = [text]
                        break
                    elif current_chapter is not None:
                        current_text.append(text)
                        
            if current_chapter is not None:
                chapter_texts[current_chapter] = " ".join(current_text)
                
        return chapter_texts

    def process_text_chunk(self, text: str) -> Dict:
        """
        Processa un chunk di testo usando Azure OpenAI per estrarre le informazioni.
        """
        chain = self.prompt | self.llm
        response = chain.invoke({"text": text})
        print("Risposta raw:", response.content)  # Debug: Stampiamo la risposta per vedere cosa arriva
        try:
            return json.loads(response.content)
        except json.JSONDecodeError:
            return {"tecniche": []}  # Ritorna una struttura vuota in caso di errore

    def extract_techniques(self, pdf_path: str, chapters: List[int]) -> Dict:
        """
        Estrae le tecniche dal PDF per i capitoli specificati.
        """
        chapter_texts = self.extract_text_from_pdf(pdf_path, chapters)
        
        results = {}
        for chapter, text in chapter_texts.items():
            print(f"Processando Capitolo {chapter}...")
            
            chunks = self.text_splitter.split_text(text)
            chapter_techniques = []
            
            for i, chunk in enumerate(chunks):
                try:
                    print(f"  Processando chunk {i+1}/{len(chunks)}")
                    extracted_info = self.process_text_chunk(chunk)
                    chapter_techniques.extend(extracted_info.get("tecniche", []))
                except Exception as e:
                    print(f"  Errore nel processare chunk {i+1}: {str(e)}")
            
            results[f"Capitolo_{chapter}"] = chapter_techniques
            
        return results

In [48]:
chapters_to_extract = [3, 4, 5]

# Inizializza e esegui l'agente
try:
    agent = TechniqueExtractionAgent(api_key, api_base)
    print("Iniziando l'estrazione delle tecniche...")
    
    techniques = agent.extract_techniques(file_path, chapters_to_extract)
    
    # Salva i risultati
    output_file = "tecniche_estratte.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(techniques, f, ensure_ascii=False, indent=2)
    
    print(f"\nEstrazione completata! Risultati salvati in {output_file}")
    
except Exception as e:
    print(f"\nErrore durante l'esecuzione: {str(e)}")

Iniziando l'estrazione delle tecniche...
Processando Capitolo 3...
  Processando chunk 1/13
Risposta raw: ```json
{
    "tecniche": [
        "Principio di Non Alterazione",
        "Marinatura",
        "Marinatura a Infusione Gravitazionale",
        "Marinatura Temporale Sincronizzata",
        "Marinatura Psionica",
        "Marinatura tramite Reazioni d'Antimateria Diluite",
        "Marinatura Sotto Zero a Polarità Inversa",
        "Affumicatura"
    ]
}
```
  Processando chunk 2/13
Risposta raw: ```json
{
    "tecniche": [
        "Marinatura Sotto Zero a Polarità Inversa",
        "Affumicatura",
        "Affumicatura a Stratificazione Quantica",
        "Affumicatura Temporale Risonante",
        "Marinatura tramite Reazioni d'Antimateria Diluite",
        "Manipolazione Molecolare Telepatica"
    ]
}
```
  Processando chunk 3/13
Risposta raw: ```json
{
    "tecniche": [
        "Affumicatura a Stratificazione Quantica",
        "Affumicatura Temporale Risonante",
        "Af